## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-30-10-38-54 +0000,bbc,Death toll from Indonesia flooding rises to 417,https://www.bbc.com/news/articles/c5y9ejley9do...
1,2025-11-30-10-12-30 +0000,bbc,Mum hosts free toy swap to help cut Christmas ...,https://www.bbc.com/news/articles/c5yq4rdrv1mo...
2,2025-11-30-10-12-23 +0000,nyt,Deadly Floods in Indonesia Leave Hundreds Missing,https://www.nytimes.com/2025/11/30/world/asia/...
3,2025-11-30-10-01-00 +0000,nyt,"India Is a Rising Power, but Its Capital Is a ...",https://www.nytimes.com/2025/11/30/world/asia/...
4,2025-11-30-10-00-56 +0000,nyt,Lebanon’s Christians Look to Pope Leo’s Visit ...,https://www.nytimes.com/2025/11/30/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
72,trump,24
141,up,9
29,pope,8
184,dead,8
30,leo,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
111,2025-11-29-20-06-00 +0000,wsj,Trump Declares That Airspace Around Venezuela ...,https://www.wsj.com/politics/national-security...,50
103,2025-11-29-20-49-01 +0000,cbc,Maduro slams 'colonial threat' as Trump says V...,https://www.cbc.ca/news/world/trump-maduro-ven...,47
122,2025-11-29-19-03-00 +0000,wsj,Why Russia and China Are Sitting Out Venezuela...,https://www.wsj.com/world/americas/why-russia-...,47
104,2025-11-29-20-42-30 +0000,wapo,Trump says airlines should consider Venezuelan...,https://www.washingtonpost.com/national-securi...,45
37,2025-11-30-06-16-39 +0000,nypost,"Rep. Troy Nehls, a Trump ally, will not seek r...",https://nypost.com/2025/11/30/us-news/texas-re...,44


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
111,50,2025-11-29-20-06-00 +0000,wsj,Trump Declares That Airspace Around Venezuela ...,https://www.wsj.com/politics/national-security...
65,39,2025-11-30-01-08-54 +0000,nypost,"Heartless son allegedly beat his parents, set ...",https://nypost.com/2025/11/29/us-news/lunatic-...
153,35,2025-11-29-13-50-50 +0000,nypost,Russia attack on Kyiv kills three as efforts t...,https://nypost.com/2025/11/29/world-news/russi...
113,34,2025-11-29-20-03-34 +0000,nypost,Pope Leo visits Turkey’s famous Blue Mosque in...,https://nypost.com/2025/11/29/world-news/pope-...
49,32,2025-11-30-03-50-26 +0000,nypost,"New York, New Jersey could get up to 5 inches ...",https://nypost.com/2025/11/29/us-news/ny-nj-co...
7,26,2025-11-30-10-00-31 +0000,nypost,Death toll in Hong Kong fire rises to 146 afte...,https://nypost.com/2025/11/30/world-news/hong-...
112,26,2025-11-29-20-05-05 +0000,nyt,"Gazans Say Israeli Forces Killed Two Children,...",https://www.nytimes.com/2025/11/29/world/middl...
13,25,2025-11-30-10-00-00 +0000,wapo,Why you may not want lower prices as much as y...,https://www.washingtonpost.com/business/2025/1...
32,23,2025-11-30-08-00-11 +0000,nyt,"Tom Stoppard, Award-Winning Playwright of Witt...",https://www.nytimes.com/2025/11/29/theater/tom...
159,21,2025-11-29-12-47-31 +0000,nypost,Mamdani-linked PAC allegedly offered Hispanic ...,https://nypost.com/2025/11/29/us-news/mamdani-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
